In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable

import os
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from datagen import ListDataset
from network import Generator, Discriminator

In [2]:
flag = {"dataset" : "densepoint",
       "weights" : "checkpoint/checkpoint_v6/ckpt_4843.pth",
       "num_pts" : 4096,
       "GPU_machine" : '0'}

In [3]:
# GPU
os.environ["CUDA_VISIBLE_DEVICES"]=flag['GPU_machine']

In [4]:
# data_loader
transform = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

testset = ListDataset(root='/root/DB/', dataset=flag['dataset'], mode="test", 
                       num_pts=flag['num_pts'], transform=transform, size=flag['size'], augmentation=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

In [5]:
# network
G = Generator(flag['size'])
G.cuda()

checkpoint = torch.load(flag['weights'])
G.load_state_dict(checkpoint['G'])

G.eval()

Generator(
  (conv1): Conv3d(3, 128, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
  (conv2): Conv3d(128, 256, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
  (conv2_bn): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv3d(256, 512, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
  (conv3_bn): BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv3d(512, 1024, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
  (conv4_bn): BatchNorm3d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv3d(1024, 1024, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
  (conv5_bn): BatchNorm3d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv4): ConvTranspose3d(1024, 1024, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
  (deconv4_bn): BatchNorm3d(1024, eps=1e-05, momentum=0.1, affine

In [1]:
for i, (x, y) in enumerate(test_loader):
    print("------------------------------------------------------------")
    x = Variable(x.cuda())
    pred = G(x)
    
    # tensor to numpy
    x = x[0].cpu().detach().numpy()
    pred = pred[0].cpu().detach().numpy()
    y = y[0].cpu().detach().numpy()
    
    # show GT
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    coord_gt, rgb_gt = voxel2pc_v2(x, y, flag['size'])
    
    xs = coord_gt[:, 0]
    ys = coord_gt[:, 1]
    zs = coord_gt[:, 2]
    s = 10
    c = rgb_gt

    ax.scatter(xs, ys, zs, s=s, c=c, marker='o', zdir='y')

    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    plt.show()
    print("sampled GT : ", coord_gt.shape[0])
    
    # show Pred
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    coord_pred, rgb_pred = voxel2pc_v2(x, pred, flag['size'], color_thread=0.45)
    
    xs = coord_pred[:, 0]
    ys = coord_pred[:, 1]
    zs = coord_pred[:, 2]
    s = 10
    c = rgb_pred

    ax.scatter(xs, ys, zs, s=s, c=c, marker='o', zdir='y')

    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    plt.show()
    print("Prediction : ", coord_pred.shape[0])
    
    if i == 10:
        break

NameError: name 'test_loader' is not defined